In [1]:
import random
import folium
from geopy.distance import geodesic
import datetime
import pandas as pd

# Define classes and functions
class TollArea:
    def __init__(self, name, entry_point, exit_point, speed_limit, toll_rates):
        self.name = name
        self.entry_point = entry_point
        self.exit_point = exit_point
        self.speed_limit = speed_limit
        self.toll_rates = toll_rates  # Dictionary for dynamic pricing

    def calculate_toll(self, distance, time_of_day, vehicle_type):
        # Dynamic pricing: increase toll rates during peak hours (e.g., 7-10 AM, 5-8 PM)
        peak_hours = [(7, 10), (17, 20)]
        base_rate = self.toll_rates.get(vehicle_type, 1.0)  # Default rate if vehicle type not found
        for start, end in peak_hours:
            if start <= time_of_day.hour < end:
                return distance * base_rate * 1.5
        return distance * base_rate

class Vehicle:
    def __init__(self, id, vehicle_type, start_location):
        self.id = id
        self.vehicle_type = vehicle_type
        self.current_location = start_location
        self.total_distance = 0
        self.total_toll = 0
        self.log = []

    def move(self, toll_area):
        distance = geodesic(toll_area.entry_point, toll_area.exit_point).km
        speed = random.uniform(toll_area.speed_limit - 10, toll_area.speed_limit + 20)  # Simulated speed
        self.current_location = toll_area.exit_point
        self.total_distance += distance
        now = datetime.datetime.now()
        toll_charge = toll_area.calculate_toll(distance, now, self.vehicle_type)
        self.total_toll += toll_charge

        speed_warning = ""
        if speed > toll_area.speed_limit:
            speed_warning = " (Speed Limit Exceeded!)"
        
        entry_data = (toll_area.name, toll_area.entry_point, now.strftime("%Y-%m-%d %H:%M:%S"))
        exit_data = (toll_area.exit_point, distance, toll_charge, speed, speed_warning)
        self.log.append((entry_data, exit_data))
        
        return distance, toll_charge, speed, speed_warning

def simulate_traffic(toll_areas, vehicles, steps):
    data = []

    for step in range(steps):
        for vehicle in vehicles:
            toll_area = random.choice(toll_areas)
            distance, toll_charge, speed, speed_warning = vehicle.move(toll_area)
            data.append([vehicle.id, vehicle.vehicle_type, toll_area.name, distance, toll_charge, speed, speed_warning, vehicle.total_distance, vehicle.total_toll])

    df = pd.DataFrame(data, columns=["Vehicle ID", "Vehicle Type", "Toll Area", "Distance (km)", "Toll Charge (₹)", "Speed (km/h)", "Speed Warning", "Total Distance (km)", "Total Toll (₹)"])
    print(df)
    return df

# Define a map area (example coordinates for a city area in Maharashtra)
map_center = [19.0760, 72.8777]  # Mumbai coordinates
map_zoom_start = 10

# Create a folium map
m = folium.Map(location=map_center, zoom_start=map_zoom_start)

# Define toll areas with entry and exit points, speed limits, and toll rates
toll_areas = [
    TollArea("Mumbai Toll", (19.0760, 72.8777), (19.2183, 73.1527), 60, {'car': 5.0, 'truck': 10.0}),
    TollArea("Pune Toll", (18.5204, 73.8567), (18.6298, 73.7997), 50, {'car': 4.0, 'truck': 8.0}),
    TollArea("Nagpur Toll", (21.1458, 79.0882), (21.2514, 79.1575), 70, {'car': 6.0, 'truck': 12.0}),
    TollArea("Nashik Toll", (19.9975, 73.7898), (20.0110, 73.7802), 55, {'car': 3.5, 'truck': 7.0}),
    TollArea("Aurangabad Toll", (19.8762, 75.3433), (19.9283, 75.3093), 65, {'car': 4.5, 'truck': 9.0})
]

# Add toll areas to the map
for area in toll_areas:
    folium.Marker(location=area.entry_point, popup=f"Entry: {area.name}").add_to(m)
    folium.Marker(location=area.exit_point, popup=f"Exit: {area.name}").add_to(m)
    folium.PolyLine(locations=[area.entry_point, area.exit_point], color='blue').add_to(m)

# Initialize vehicles with different types
vehicles = [
    Vehicle(1, 'car', map_center),
    Vehicle(2, 'truck', map_center),
    Vehicle(3, 'car', map_center),
    Vehicle(4, 'truck', map_center),
    Vehicle(5, 'car', map_center)
]

# Run the simulation for 10 steps and get the dataframe
df = simulate_traffic(toll_areas, vehicles, 10)

# Add vehicle entry and exit details to the map
for vehicle in vehicles:
    for entry_data, exit_data in vehicle.log:
        toll_name, entry_point, entry_time = entry_data
        exit_point, distance, toll_charge, speed, speed_warning = exit_data
        
        folium.Marker(location=entry_point, popup=f"Vehicle {vehicle.id} Entry: {toll_name} at {entry_time}").add_to(m)
        folium.Marker(location=exit_point, popup=f"Vehicle {vehicle.id} Exit: {toll_name}, Toll: ₹{toll_charge:.2f}, Speed: {speed:.2f} km/h{speed_warning}").add_to(m)

    # Mark the final position of each vehicle
    folium.Marker(location=vehicle.current_location, popup=f"Vehicle {vehicle.id} ({vehicle.vehicle_type}), Total Toll: ₹{vehicle.total_toll:.2f}").add_to(m)

# Save the map to an HTML file
m.save("toll_areas_mapf.html")


    Vehicle ID Vehicle Type        Toll Area  Distance (km)  Toll Charge (₹)  \
0            1          car  Aurangabad Toll       6.777944        30.500747   
1            2        truck        Pune Toll      13.521562       108.172495   
2            3          car      Nashik Toll       1.800763         6.302670   
3            4        truck        Pune Toll      13.521562       108.172495   
4            5          car      Nashik Toll       1.800763         6.302670   
5            1          car      Nagpur Toll      13.728763        82.372580   
6            2        truck      Mumbai Toll      32.940052       329.400524   
7            3          car  Aurangabad Toll       6.777944        30.500747   
8            4        truck  Aurangabad Toll       6.777944        61.001494   
9            5          car      Nashik Toll       1.800763         6.302670   
10           1          car  Aurangabad Toll       6.777944        30.500747   
11           2        truck      Mumbai 